In [1]:
import os
os.chdir("../../")

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

In [3]:
!Rscript Toy.R

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    

In [4]:
counts_df = pd.read_csv("./data/toy/toy.csv")
counts_df = counts_df.transpose()
counts_df.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene4991,Gene4992,Gene4993,Gene4994,Gene4995,Gene4996,Gene4997,Gene4998,Gene4999,Gene5000
Cell1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,4
Cell2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
Cell3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Cell4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
Cell5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2


In [5]:
batches_df = pd.read_csv("./data/toy/toy_batch.csv")
batches_df['x'].value_counts()

Batch6    4000
Batch5    4000
Batch2    4000
Batch4    4000
Batch7    4000
Batch8    4000
Batch1    4000
Batch9    4000
Batch3    4000
Name: x, dtype: int64

In [6]:
celltypes_df = pd.read_csv("./data/toy/toy_celltype.csv")
celltypes_df['x'].value_counts()

Group3    7313
Group2    7257
Group1    7171
Group5    7137
Group4    7122
Name: x, dtype: int64

In [8]:
counts_df.shape, batches_df.shape, celltypes_df.shape

((36000, 5000), (36000, 1), (36000, 1))

In [9]:
counts_adata = sc.AnnData(X=counts_df.values)
counts_adata.obs['study'] = batches_df.values
counts_adata.obs['celltype'] = celltypes_df.values
counts_adata.var_names = counts_df.columns
counts_adata

AnnData object with n_obs × n_vars = 36000 × 5000 
    obs: 'study', 'celltype'

In [10]:
counts_adata.write_h5ad("./data/toy/toy_count.h5ad")

... storing 'study' as categorical
... storing 'celltype' as categorical


In [11]:
import scnet as sn

Using TensorFlow backend.


In [18]:
normalized_adata = counts_adata.copy()

In [19]:
normalized_adata = sn.tl.normalize(normalized_adata, 'study', target_sum=1e4)
normalized_adata

Using 8 HVGs from full intersect set
Using 39 HVGs from n_batch-1 set
Using 152 HVGs from n_batch-2 set
Using 407 HVGs from n_batch-3 set
Using 730 HVGs from n_batch-4 set
Using 664 HVGs from n_batch-5 set
Using 2000 HVGs


AnnData object with n_obs × n_vars = 36000 × 2000 
    obs: 'study', 'celltype', 'n_counts', 'size_factors'
    var: 'n_counts'
    uns: 'log1p'

In [20]:
normalized_adata.X.min(), normalized_adata.X.max()

(0.0, 13.293877)

In [21]:
normalized_adata.write_h5ad("./data/toy/toy_normalized.h5ad")